In [1]:
# Imports
import das4whales as dw
import scipy.signal as sp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cv2

In [2]:
# Matplotlib settings
plt.rcParams['font.size'] = 20

In [3]:
# The dataset of this example is constituted of 60s time series along  66 km of cable
url_before = 'http://piweb.ooirsn.uw.edu/das/data/Optasense/NorthCable/TransmitFiber/'\
        'North-C1-LR-P1kHz-GL50m-Sp2m-FS200Hz_2021-11-03T15_06_51-0700/'\
        'North-C1-LR-P1kHz-GL50m-Sp2m-FS200Hz_2021-11-04T015902Z.h5'

url = 'http://piweb.ooirsn.uw.edu/das/data/Optasense/NorthCable/TransmitFiber/'\
        'North-C1-LR-P1kHz-GL50m-Sp2m-FS200Hz_2021-11-03T15_06_51-0700/'\
        'North-C1-LR-P1kHz-GL50m-Sp2m-FS200Hz_2021-11-04T020002Z.h5'

url_next = 'http://piweb.ooirsn.uw.edu/das/data/Optasense/NorthCable/TransmitFiber/'\
        'North-C1-LR-P1kHz-GL50m-Sp2m-FS200Hz_2021-11-03T15_06_51-0700/'\
        'North-C1-LR-P1kHz-GL50m-Sp2m-FS200Hz_2021-11-04T020102Z.h5'

url_south = 'http://piweb.ooirsn.uw.edu/das/data/Optasense/SouthCable/TransmitFiber/'\
        'South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/'\
        'South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-04T020014Z.h5'

filepath = dw.data_handle.dl_file(url_south)

South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-04T020014Z.h5 already stored locally


In [4]:
# Read HDF5 files and access metadata
# Get the acquisition parameters for the data folder
metadata = dw.data_handle.get_acquisition_parameters(filepath, interrogator='optasense')
fs, dx, nx, ns, gauge_length, scale_factor = metadata["fs"], metadata["dx"], metadata["nx"], metadata["ns"], metadata["GL"], metadata["scale_factor"]

print(f'Sampling frequency: {metadata["fs"]} Hz')
print(f'Channel spacing: {metadata["dx"]} m')
print(f'Gauge length: {metadata["GL"]} m')
print(f'File duration: {metadata["ns"] / metadata["fs"]} s')
print(f'Cable max distance: {metadata["nx"] * metadata["dx"]/1e3:.1f} km')
print(f'Number of channels: {metadata["nx"]}')
print(f'Number of time samples: {metadata["ns"]}')

Sampling frequency: 200.0 Hz
Channel spacing: 2.0419046878814697 m
Gauge length: 51.0476188659668 m
File duration: 60.0 s
Cable max distance: 97.0 km
Number of channels: 47500
Number of time samples: 12000


In [5]:
# South cable
# selected_channels_m = [20000, 80000, 5]  # list of values in meters corresponding to the starting,
#                                           # ending and step wanted channels along the FO Cable
#                                           # selected_channels_m = [ChannelStart_m, ChannelStop_m, ChannelStep_m]
#                                           # in meters

# North cable
selected_channels_m = [12000, 80000, 3]  # list of values in meters corresponding to the starting,
                                          # ending and step wanted channels along the FO Cable
                                          # selected_channels_m = [ChannelStart_m, ChannelStop_m, ChannelStep_m]
                                          # in meters

selected_channels = [int(selected_channels_m // dx) for selected_channels_m in
                     selected_channels_m]  # list of values in channel number (spatial sample) corresponding to the starting, ending and step wanted
                                           # channels along the FO Cable
                                           # selected_channels = [ChannelStart, ChannelStop, ChannelStep] in channel
                                           # numbers

print('Begin channel #:', selected_channels[0], 
      ', End channel #: ',selected_channels[1], 
      ', step: ',selected_channels[2], 
      'equivalent to ',selected_channels[2]*dx,' m')

Begin channel #: 5876 , End channel #:  39179 , step:  1 equivalent to  2.0419046878814697  m


In [6]:
tr, time, dist, fileBeginTimeUTC = dw.data_handle.load_das_data(filepath, selected_channels, metadata)

In [7]:
print(np.shape(tr))

(33303, 12000)


In [8]:
# Create the f-k filter 
# includes band-pass filter trf = sp.sosfiltfilt(sos_bpfilter, tr, axis=1) 

# fk_filter = dw.dsp.hybrid_ninf_gs_filter_design((tr.shape[0],tr.shape[1]), selected_channels, dx, fs, c_min=1450, c_max=3300, fmin=14, fmax=30, display_filter=False)
# fk_filter_noise = dw.dsp.hybrid_ninf_gs_filter_design((tr.shape[0],tr.shape[1]), selected_channels, dx, fs, c_min=1450, c_max=3500, fmin=35, fmax=51, display_filter=False)

fk_filter = dw.dsp.hybrid_ninf_filter_design((tr.shape[0],tr.shape[1]), selected_channels, dx, fs, cs_min=1400, cp_min=1480, cp_max=3300, cs_max=3500, fmin=14, fmax=30, display_filter=False)
fk_filter_noise = dw.dsp.hybrid_ninf_filter_design((tr.shape[0],tr.shape[1]), selected_channels, dx, fs, cs_min=1400, cp_min=1480, cp_max=3300, cs_max=3500, fmin=35, fmax=51, display_filter=False)

In [ ]:

# Print the compression ratio given by the sparse matrix usage
dw.tools.disp_comprate(fk_filter)

# Apply the hybrid f-k filter to the data, returns spatio-temporal strain matrix
trf_fk = dw.dsp.fk_filter_sparsefilt(tr, fk_filter, tapering=True)
noise = dw.dsp.fk_filter_sparsefilt(tr, fk_filter_noise, tapering=True)

# Delete the original data to free memory
del tr

The size of the sparse filter is 1.7178 Gib
The size of the dense filter is 2.98 Gib
The compression ratio is 1.73 (42.3 %)


In [ ]:
# Compute the cross-correlograms

trf_fk = dw.dsp.normalize_std(trf_fk)

HF_note = dw.detect.gen_hyperbolic_chirp(17.8, 28.8, 0.68, fs)
HF_note = np.hanning(len(HF_note)) * HF_note

LF_note = dw.detect.gen_hyperbolic_chirp(14.7, 21.8, 0.78, fs)
LF_note = np.hanning(len(LF_note)) * LF_note

nmf_m_HF = dw.detect.calc_nmf_correlogram(trf_fk, HF_note)
nmf_m_LF = dw.detect.calc_nmf_correlogram(trf_fk, LF_note)

nmf_m_HF = dw.dsp.normalize_std(nmf_m_HF)
nmf_m_LF = dw.dsp.normalize_std(nmf_m_LF)

del trf_fk

In [ ]:
def moving_average(signal, window_size):
    return np.convolve(signal, np.ones(window_size) / window_size, mode='same')


def moving_average_matrix(matrix, window_size):
    return np.array([moving_average(matrix[i, :], window_size) for i in range(matrix.shape[0])])

window_size = 100  # Define the window size for the moving average
noise = dw.dsp.normalize_std(noise)

# noise_avg = moving_average_matrix(abs(sp.hilbert(noise)), window_size)
noise_avg = moving_average_matrix(abs(sp.hilbert(noise)), window_size)
del noise


In [ ]:
SNR_hf = 20 * np.log10(abs(sp.hilbert(nmf_m_HF, axis=1)) / abs(sp.hilbert(noise_avg, axis=1)))
SNR_lf = 20 * np.log10(abs(sp.hilbert(nmf_m_LF, axis=1)) / abs(sp.hilbert(noise_avg, axis=1)))

# Smooth the SNR with a gaussian filter and normalize it
th_SNR_hf = cv2.GaussianBlur(SNR_hf, (9, 73), 0)
th_SNR_lf = cv2.GaussianBlur(SNR_lf, (9, 73), 0)


In [ ]:
# Threshold the SNR matrix in an efficient way, keeping the original matrix
th_SNR_hf = np.where(th_SNR_hf < 0, 0, th_SNR_hf)
th_SNR_lf = np.where(th_SNR_hf < 0, 0, th_SNR_lf)

del fk_filter, nmf_m_HF, nmf_m_LF

In [ ]:
dw.plot.snr_matrix(th_SNR_hf, time, dist, 20, fileBeginTimeUTC, title='mf detect: HF')
dw.plot.snr_matrix(th_SNR_lf, time, dist, 20, fileBeginTimeUTC, title ='mf detect: LF')

In [ ]:
# Purge memory
del SNR_hf, SNR_lf

In [ ]:
# Detect the events
from tqdm import tqdm

ipi = 3 # Inter pulse interval in seconds
th = 0. # Threshold for the peak detection in dB

peaks_indexes_HF = []
peaks_indexes_LF = []

for corr in tqdm(th_SNR_hf, desc="Processing corr_m"):
    peaks_indexes,_ = sp.find_peaks(corr, distance = ipi * fs, height=th)
    peaks_indexes_HF.append(peaks_indexes)

for corr in tqdm(th_SNR_lf, desc="Processing corr_m"):
    peaks_indexes,_ = sp.find_peaks(corr, distance = ipi * fs, height=th)  
    peaks_indexes_LF.append(peaks_indexes)

# # Convert the list of array to tuple format
peaks_indexes_tp_HF = dw.detect.convert_pick_times(peaks_indexes_HF)
peaks_indexes_tp_LF = dw.detect.convert_pick_times(peaks_indexes_LF)

In [ ]:
# Try delayed histogram on the detections with 5 km distance window
plt.figure(figsize=(12,10))
plt.scatter(peaks_indexes_tp_HF[1] / fs, (peaks_indexes_tp_HF[0] * selected_channels[2] + selected_channels[0]) * dx /1e3, color='tab:blue', marker='.', label='HF_note', s=0.5)
plt.xlabel('Time (s)')
plt.ylabel('Distance (km)')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(peaks_indexes_tp_LF[1] / fs, (peaks_indexes_tp_LF[0] * selected_channels[2] + selected_channels[0]) * dx /1e3, color='tab:red', marker='.', label='HF_note', s=0.5)
plt.xlabel('Time (s)')
plt.ylabel('Distance (km)')
plt.show()